In [12]:
!pip install moviepy docopt

  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
   ---------------------------------------- 0.0/31.2 MB ? eta -:--:--
   -- ------------------------------------- 1.8/31.2 MB 9.1 MB/s eta 0:00:04
   ----- ---------------------------------- 4.5/31.2 MB 10.3 MB/s eta 0:00:03
   -------- ------------------------------- 6.3/31.2 MB 10.2 MB/s eta 0:00:03
   ---------- ----------------------------- 8.4/31.2 MB 9.8 MB/s eta 0:00:03
   ----------------- ---------------------- 13.9/31.2 MB 13.2 MB/s eta 0:00:02
   --------------------- ------------------ 17.0/31.2 MB 13.8 MB/s eta 0:00:02
   ------------------------ --------------- 19.1/31.2 MB 12.8 MB/s eta 0:00:01
   ---------------------------- ----------- 22.0/31.2 MB 13.1 MB/s eta 0:00:01
   -------------------------------- ------- 25.4/31.2 MB 13.4 MB/s eta 0:00:01
   -------------------------------------- - 29.9/31.2 MB 14.1 MB/s eta 0:00:01
   ---------------------------------------- 

## Camera Caliberation

In [1]:
import numpy as np
import cv2
import glob
import matplotlib.image as mpimg
import matplotlib.pyplot as plt

class CameraCalibration():

    def __init__(self, dir, w, h, debug=False):
      
        fnames = glob.glob("{}/*".format(dir))
        objpt = []
        imgpt = []

        objp = np.zeros((w*h, 3), np.float32)
        objp[:,:2] = np.mgrid[0:w, 0:h].T.reshape(-1, 2)

        for f in fnames:
            img = mpimg.imread(f)

            gray = cv2.cvtColor(img, cv2.COLOR_RGB2GRAY)

            ret, corners = cv2.findChessboardCorners(img, (w, h))
            if ret:
                imgpt.append(corners)
                objpt.append(objp)

        shape = (img.shape[1], img.shape[0])
        ret, self.mtx, self.dist, _, _ = cv2.calibrateCamera(objpt, imgpt, shape, None, None)

        if not ret:
            raise Exception("calibration error")

    def undistort(self, img):
        
        gray = cv2.cvtColor(img, cv2.COLOR_RGB2GRAY)
        return cv2.undistort(img, self.mtx, self.dist, None, self.mtx)


## Thresholding

In [2]:
import cv2
import numpy as np

def threshold_rel(img, lo, hi):
    vmin = np.min(img)
    vmax = np.max(img)

    vlo = vmin + (vmax - vmin) * lo
    vhi = vmin + (vmax - vmin) * hi
    return np.uint8((img >= vlo) & (img <= vhi)) * 255

def threshold_abs(img, lo, hi):
    return np.uint8((img >= lo) & (img <= hi)) * 255

class Thresholding:
   
    def __init__(self):
        pass

    def forward(self, img):
       
        hls = cv2.cvtColor(img, cv2.COLOR_RGB2HLS)
        hsv = cv2.cvtColor(img, cv2.COLOR_RGB2HSV)
        h_c = hls[:,:,0]
        l_c = hls[:,:,1]
        s_c = hls[:,:,2]
        v_c = hsv[:,:,2]

        right = threshold_rel(l_c, 0.8, 1.0)
        right[:,:750] = 0

        left = threshold_abs(h_c, 20, 30)
        left &= threshold_rel(v_c, 0.7, 1.0)
        left[:,550:] = 0

        img2 = left | right

        return img2


## Perspective Transformation

In [3]:
import cv2
import numpy as np

class PerspectiveTransformation:
    
    def __init__(self):
        self.src = np.float32([(550, 460),     
                               (150, 720),     
                               (1200, 720),    
                               (770, 460)])    
        self.dst = np.float32([(100, 0),
                               (100, 720),
                               (1100, 720),
                               (1100, 0)])
        self.M = cv2.getPerspectiveTransform(self.src, self.dst)
        self.M_inv = cv2.getPerspectiveTransform(self.dst, self.src)

    def forward(self, img, img_size=(1280, 720), flags=cv2.INTER_LINEAR):
      
        return cv2.warpPerspective(img, self.M, img_size, flags=flags)

    def backward(self, img, img_size=(1280, 720), flags=cv2.INTER_LINEAR):
       
        return cv2.warpPerspective(img, self.M_inv, img_size, flags=flags)


## Lane Lines

In [4]:
import cv2
import numpy as np
import matplotlib.image as mpimg

def hist(img):
    b_half = img[img.shape[0]//2:,:]
    return np.sum(b_half, axis=0)

class LaneLines:
    
    def __init__(self):
       
        self.l_f = None
        self.r_f = None
        self.binary = None
        self.nonzero = None
        self.nonzerox = None
        self.nonzeroy = None
        self.clear_visibility = True
        self.dir = []
        self.l_img = mpimg.imread('left_turn.png')
        self.r_img = mpimg.imread('right_turn.png')
        self.s_img = mpimg.imread('straight.png')
        self.l_img = cv2.normalize(src=self.l_img, dst=None, alpha=0, beta=255, norm_type=cv2.NORM_MINMAX, dtype=cv2.CV_8U)
        self.r_img = cv2.normalize(src=self.r_img, dst=None, alpha=0, beta=255, norm_type=cv2.NORM_MINMAX, dtype=cv2.CV_8U)
        self.s_img = cv2.normalize(src=self.s_img, dst=None, alpha=0, beta=255, norm_type=cv2.NORM_MINMAX, dtype=cv2.CV_8U)

        self.nwindows = 9
        self.margin = 100
        self.minpix = 50

    def forward(self, img):
       
        self.extract_features(img)
        return self.fit_poly(img)

    def pixels_in_window(self, center, margin, height):
        
        topleft = (center[0]-margin, center[1]-height//2)
        bottomright = (center[0]+margin, center[1]+height//2)

        condx = (topleft[0] <= self.nonzerox) & (self.nonzerox <= bottomright[0])
        condy = (topleft[1] <= self.nonzeroy) & (self.nonzeroy <= bottomright[1])
        return self.nonzerox[condx&condy], self.nonzeroy[condx&condy]

    def extract_features(self, img):
        
        self.img = img
        self.window_height = int(img.shape[0]//self.nwindows)
        self.nonzero = img.nonzero()
        self.nonzerox = np.array(self.nonzero[1])
        self.nonzeroy = np.array(self.nonzero[0])

    def find_lane_pixels(self, img):
       
        assert(len(img.shape) == 2)

        out_img = np.dstack((img, img, img))

        histogram = hist(img)
        midpoint = histogram.shape[0]//2
        leftx_base = np.argmax(histogram[:midpoint])
        rightx_base = np.argmax(histogram[midpoint:]) + midpoint

        leftx_current = leftx_base
        rightx_current = rightx_base
        y_current = img.shape[0] + self.window_height//2

        leftx, lefty, rightx, righty = [], [], [], []

        for _ in range(self.nwindows):
            y_current -= self.window_height
            center_left = (leftx_current, y_current)
            center_right = (rightx_current, y_current)

            good_left_x, good_left_y = self.pixels_in_window(center_left, self.margin, self.window_height)
            good_right_x, good_right_y = self.pixels_in_window(center_right, self.margin, self.window_height)

            leftx.extend(good_left_x)
            lefty.extend(good_left_y)
            rightx.extend(good_right_x)
            righty.extend(good_right_y)

            if len(good_left_x) > self.minpix:
                leftx_current = int(np.mean(good_left_x))
            if len(good_right_x) > self.minpix:
                rightx_current = int(np.mean(good_right_x))

        return leftx, lefty, rightx, righty, out_img

    def fit_poly(self, img):
       

        leftx, lefty, rightx, righty, out_img = self.find_lane_pixels(img)

        if len(lefty) > 1500:
            self.l_f = np.polyfit(lefty, leftx, 2)
        if len(righty) > 1500:
            self.r_f = np.polyfit(righty, rightx, 2)

        maxy = img.shape[0] - 1
        mih = img.shape[0] // 3
        if len(lefty):
            maxy = max(maxy, np.max(lefty))
            mih = min(mih, np.min(lefty))

        if len(righty):
            maxy = max(maxy, np.max(righty))
            mih = min(mih, np.min(righty))

        ploty = np.linspace(mih, maxy, img.shape[0])

        l_fx = self.l_f[0]*ploty**2 + self.l_f[1]*ploty + self.l_f[2]
        r_fx = self.r_f[0]*ploty**2 + self.r_f[1]*ploty + self.r_f[2]

        for i, y in enumerate(ploty):
            l = int(l_fx[i])
            r = int(r_fx[i])
            y = int(y)
            cv2.line(out_img, (l, y), (r, y), (0, 255, 0))

        lR, rR, pos = self.measure_curvature()

        return out_img

    def plot(self, out_img):
        np.set_printoptions(precision=6, suppress=True)
        lR, rR, pos = self.measure_curvature()

        value = None
        if abs(self.l_f[0]) > abs(self.r_f[0]):
            value = self.l_f[0]
        else:
            value = self.r_f[0]

        if abs(value) <= 0.00015:
            self.dir.append('F')
        elif value < 0:
            self.dir.append('L')
        else:
            self.dir.append('R')

        if len(self.dir) > 10:
            self.dir.pop(0)

        W = 400
        H = 500
        widget = np.copy(out_img[:H, :W])
        widget //= 2
        widget[0,:] = [0, 0, 255]
        widget[-1,:] = [0, 0, 255]
        widget[:,0] = [0, 0, 255]
        widget[:,-1] = [0, 0, 255]
        out_img[:H, :W] = widget

        direction = max(set(self.dir), key = self.dir.count)
        msg = "Keep Straight Ahead"
        curvature_msg = "Curvature = {:.0f} m".format(min(lR, rR))
        if direction == 'L':
            y, x = self.l_img[:,:,3].nonzero()
            out_img[y, x-100+W//2] = self.l_img[y, x, :3]
            msg = "Left Curve Ahead"
        if direction == 'R':
            y, x = self.r_img[:,:,3].nonzero()
            out_img[y, x-100+W//2] = self.r_img[y, x, :3]
            msg = "Right Curve Ahead"
        if direction == 'F':
            y, x = self.s_img[:,:,3].nonzero()
            out_img[y, x-100+W//2] = self.s_img[y, x, :3]

        cv2.putText(out_img, msg, org=(10, 240), fontFace=cv2.FONT_HERSHEY_SIMPLEX, fontScale=1, color=(255, 255, 255), thickness=2)
        if direction in 'LR':
            cv2.putText(out_img, curvature_msg, org=(10, 280), fontFace=cv2.FONT_HERSHEY_SIMPLEX, fontScale=1, color=(255, 255, 255), thickness=2)

        cv2.putText(
            out_img,
            "Good Lane Keeping",
            org=(10, 400),
            fontFace=cv2.FONT_HERSHEY_SIMPLEX,
            fontScale=1.2,
            color=(0, 255, 0),
            thickness=2)

        cv2.putText(
            out_img,
            "Vehicle is {:.2f} m away from center".format(pos),
            org=(10, 450),
            fontFace=cv2.FONT_HERSHEY_SIMPLEX,
            fontScale=0.66,
            color=(255, 255, 255),
            thickness=2)

        return out_img

    def measure_curvature(self):
        ym = 30/720
        xm = 3.7/700

        l_f = self.l_f.copy()
        r_f = self.r_f.copy()
        y_eval = 700 * ym

        left_curveR =  ((1 + (2*l_f[0] *y_eval + l_f[1])**2)**1.5)  / np.absolute(2*l_f[0])
        right_curveR = ((1 + (2*r_f[0]*y_eval + r_f[1])**2)**1.5) / np.absolute(2*r_f[0])

        xl = np.dot(self.l_f, [700**2, 700, 1])
        xr = np.dot(self.r_f, [700**2, 700, 1])
        pos = (1280//2 - (xl+xr)//2)*xm
        return left_curveR, right_curveR, pos


## main body

In [ ]:
import numpy as np
import matplotlib.image as mpimg
import cv2
import docopt
from IPython.display import HTML
from IPython.core.display import Video
from moviepy.editor import VideoFileClip


class FindLaneLines:
    def __init__(self):
       
        self.calibration = CameraCalibration('camera_cal', 9, 6)
        self.thresholding = Thresholding()
        self.transform = PerspectiveTransformation()
        self.lanelines = LaneLines()

    def forward(self, img):
        out_img = np.copy(img)
        img = self.calibration.undistort(img)
        img = self.transform.forward(img)
        img = self.thresholding.forward(img)
        img = self.lanelines.forward(img)
        img = self.transform.backward(img)

        out_img = cv2.addWeighted(out_img, 1, img, 0.6, 0)
        out_img = self.lanelines.plot(out_img)
        return out_img

    def process_image(self, input_path, output_path):
        img = mpimg.imread(input_path)
        out_img = self.forward(img)
        mpimg.imsave(output_path, out_img)

    def process_video(self, input_path, output_path):
        clip = VideoFileClip(input_path)
        out_clip = clip.fl_image(self.forward)
        out_clip.write_videofile(output_path, audio=False)

def main():
    findLaneLines = FindLaneLines()
    findLaneLines.process_video("challenge_video.mp4","output.mp4")

if __name__ == "__main__":
    main()